## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-16-21-50-40 +0000,nypost,Trump fumes at ‘politically correct’ Indiana s...,https://nypost.com/2025/11/16/us-news/trump-fu...
1,2025-11-16-21-37-51 +0000,bbc,Leader of notorious Ecuador drugs gang capture...,https://www.bbc.com/news/articles/c93dz9zly83o...
2,2025-11-16-21-35-49 +0000,nypost,"LIRR worker axed, others on the block after bo...",https://nypost.com/2025/11/16/us-news/lirr-wor...
3,2025-11-16-21-34-40 +0000,nypost,Netanyahu vows that Hamas will be disarmed no ...,https://nypost.com/2025/11/16/world-news/netan...
4,2025-11-16-21-32-28 +0000,bbc,Marjorie Taylor Greene doubles down on Epstein...,https://www.bbc.com/news/articles/cdeg8z0j2j2o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,16
44,epstein,9
119,government,8
97,killed,6
102,year,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...,51
79,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...,49
4,2025-11-16-21-32-28 +0000,bbc,Marjorie Taylor Greene doubles down on Epstein...,https://www.bbc.com/news/articles/cdeg8z0j2j2o...,49
107,2025-11-16-08-12-45 +0000,bbc,Feud erupts between Trump and ally Marjorie Ta...,https://www.bbc.com/news/articles/c9wv4dx05q5o...,49
69,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,48


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,51,2025-11-16-20-26-00 +0000,wsj,President Trump is making a last-ditch push to...,https://www.wsj.com/politics/policy/trump-play...
27,38,2025-11-16-20-23-31 +0000,nypost,Jets cornerback Kris Boyd’s pal gives update o...,https://nypost.com/2025/11/16/sports/jets-corn...
132,32,2025-11-16-01-42-19 +0000,cbc,Anti-government protesters clash with riot pol...,https://www.cbc.ca/news/world/gen-z-protest-me...
140,29,2025-11-15-23-12-47 +0000,latimes,"5-year-old girl missing, father dead after lar...",https://www.latimes.com/california/story/2025-...
79,24,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...
59,22,2025-11-16-18-03-28 +0000,nypost,Schoolgirl murdered in her own bed by intruder...,https://nypost.com/2025/11/16/world-news/schoo...
85,22,2025-11-16-13-28-00 +0000,wsj,New Rare-Earths Plant in Europe Shows How Toug...,https://www.wsj.com/world/europe/a-new-rare-ea...
120,21,2025-11-16-03-51-02 +0000,missionlocal,‘Mole to Die For’: Mission District chefs cook...,https://missionlocal.org/2025/11/mole-to-die-f...
34,21,2025-11-16-19-48-41 +0000,nypost,India arrests Kashmir resident over Delhi car ...,https://nypost.com/2025/11/16/world-news/india...
31,17,2025-11-16-20-07-45 +0000,nypost,Treasury Secretary Scott Bessent warns of ‘per...,https://nypost.com/2025/11/16/us-news/trump-of...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
